In [ ]:
import pandas as pd
from sklearn.datasets import fetch_kddcup99

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

import seaborn

In [ ]:
#Données
X, y = fetch_kddcup99(return_X_y  = True)

In [ ]:
#supprime les colonnes textuelles 
# l'objectif est de définir un modele de détection qui est agnostic au type de connection (qui fonctionne pour tout type de connection)
X = pd.DataFrame(X).iloc[:,4:]

In [ ]:
# y new : reclassifier en normal (valeur 0) ou risqué (valeur 1) pour détecter tout type de risque (non pas seulement les types qui sont dans le train data)
y_new = []
for i in y:
  if i == b'normal.':
    y_new.append(0)
  else:
    y_new.append(1)

In [ ]:
seaborn.countplot(y_new)

In [ ]:
#couper les donnees en 3 : train (pour apprendre les modeles), validation (pour comparer different modeles, test (pour mesurer la performance du modele choisi)
X_train, X_test, y_train, y_test = train_test_split(X, y_new, test_size=0.33, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [9]:
#Random Forest
parameters_RF = {'max_features':[0.85, 0.9, 0.95]}
RF = GridSearchCV(RandomForestClassifier(), parameters_RF, cv=5)
RF.fit(X_train, y_train)

C:\Users\mon ordinateur\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\mon ordinateur\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\mon ordinateur\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\mon ordinateur\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\mon ordinateur\Anaconda3\lib\si

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_features': [0.85, 0.9, 0.95]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
RF_accuracy = roc_auc_score(y_val, RF.predict(X_val))
RF_accuracy

In [ ]:
RF.feature_importances_

In [ ]:
# Decision Tree
parameters_DT = {'max_features':[0.85, 0.9, 0.95], 'min_samples_split': [2, 3, 4, 5, 10]}
DT = GridSearchCV(DecisionTreeClassifier(), parameters_DT, cv=5)
DT.fit(X_train, y_train)

In [ ]:
DT_accuracy = roc_auc_score(y_val, DT.predict(X_val))


In [ ]:
DT_accuracy

In [ ]:
# Regression Logistique
parameters_LR = {'C':[0.01, 0.1, 1.0]}
LR = GridSearchCV(LogisticRegression(), parameters_LR, cv=5)
LR.fit(X_train, y_train)

In [ ]:
LR_accuracy = roc_auc_score(y_val, LR.predict(X_val))
LR_accuracy

In [ ]:
# SVM
parameters_SVM = {'C':[0.01, 0.1, 1.0], 'kernel': ['linear']}
SVM = GridSearchCV(SVC(), parameters_SVM, cv=5)
SVM.fit(X_train, y_train)

In [ ]:
SVM_accuracy = roc_auc_score(y_val, SVM.predict(X_val))
SVM_accuracy

In [ ]:
# ici les trois accuracy sur la base de validation
RF_accuracy, DT_accuracy, LR_accuracy

In [ ]:
# le meilleur modele est a executer ici: cela donne le vrai resultat
roc_auc_score(y_test, RF.predict(X_test)) # si cest le RF (random forect)
roc_auc_score(y_test, DT.predict(X_test)) # si cest le DT (decision tree)
roc_auc_score(y_test, LR.predict(X_test)) # si cest le LR (regressioh lineaire)